<a href="https://colab.research.google.com/github/bhavanapagadala/hackton/blob/main/AI_Disaster.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
!pip install folium geopy gradio


In [4]:
pip install folium geopy gradio

In [5]:
import os
os.system('pip install folium geopy gradio')

0

In [6]:
%pip install gradio folium geopy

In [19]:
import gradio as gr
import matplotlib.pyplot as plt
import folium
from folium.plugins import Search
from geopy.distance import geodesic
import random
import base64

# Disaster data
disaster_data = [
    {"city": "Delhi", "lat": 28.6139, "lon": 77.2090, "severity": "High", "safe_zone": "Indira Stadium", "safe_lat": 28.6330, "safe_lon": 77.2185, "rescue_number": "011-100"},
    {"city": "Mumbai", "lat": 19.0760, "lon": 72.8777, "severity": "Critical", "safe_zone": "Wankhede Grounds", "safe_lat": 19.0000, "safe_lon": 72.8200, "rescue_number": "022-100"},
    {"city": "Chennai", "lat": 13.0827, "lon": 80.2707, "severity": "Moderate", "safe_zone": "Marina Shelter", "safe_lat": 13.0400, "safe_lon": 80.2800, "rescue_number": "044-100"},
    {"city": "Kolkata", "lat": 22.5726, "lon": 88.3639, "severity": "Severe", "safe_zone": "Netaji Stadium", "safe_lat": 22.5600, "safe_lon": 88.3700, "rescue_number": "033-100"},
    {"city": "Coimbatore", "lat": 11.0168, "lon": 76.9558, "severity": "Moderate", "safe_zone": "PSG Auditorium", "safe_lat": 11.018, "safe_lon": 76.9600, "rescue_number": "0422-100"},
    {"city": "Kottur", "lat": 10.9331, "lon": 76.9565, "severity": "Severe", "safe_zone": "Kottur School Ground", "safe_lat": 10.936, "safe_lon": 76.9588, "rescue_number": "0462-100"}
]

# Emergency Points
emergency_points = [
    {"type": "Rescue Center", "name": "National Rescue HQ", "lat": 28.65, "lon": 77.22},
    {"type": "Hospital", "name": "AIIMS Delhi", "lat": 28.5672, "lon": 77.2100},
    {"type": "Hospital", "name": "KEM Hospital Mumbai", "lat": 18.968, "lon": 72.8258},
    {"type": "Rescue Center", "name": "Mumbai Coast Guard", "lat": 19.04, "lon": 72.82},
    {"type": "Hospital", "name": "Apollo Chennai", "lat": 13.0644, "lon": 80.2616},
    {"type": "Rescue Center", "name": "Chennai Rescue Ops", "lat": 13.1, "lon": 80.28},
    {"type": "Hospital", "name": "Fortis Kolkata", "lat": 22.55, "lon": 88.35},
    {"type": "Rescue Center", "name": "Kolkata Flood Relief", "lat": 22.58, "lon": 88.38}
]

severity_colors = {
    "Critical": "darkred", "Severe": "orange", "High": "red",
    "Moderate": "blue", "Low": "green"
}
severity_radius = {
    "Critical": 10000, "Severe": 7000, "High": 5000,
    "Moderate": 3000, "Low": 2000
}

# Pie Chart
def generate_pie_chart(resources):
    fig, ax = plt.subplots()
    ax.pie(list(resources.values()), labels=list(resources.keys()), autopct='%1.1f%%', startangle=90)
    ax.axis('equal')
    path = "pie_chart.png"
    plt.tight_layout()
    plt.savefig(path)
    plt.close()
    return path

# Bar Chart
def generate_bar_chart(resources):
    fig, ax = plt.subplots(figsize=(6, 4))
    colors = {
        "Medical": "darkred", "Food": "orange",
        "Rescue Teams": "red", "Shelter": "blue"
    }
    ax.bar(
        resources.keys(),
        resources.values(),
        color=[colors.get(k, "#3498db") for k in resources.keys()]
    )
    ax.set_ylabel('Quantity', fontsize=12)
    ax.set_title('Resource Allocation', fontsize=14, color="#333")
    ax.grid(axis='y', linestyle='--', alpha=0.7)
    plt.xticks(rotation=15)
    path = "bar_chart.png"
    plt.tight_layout()
    plt.savefig(path)
    plt.close()
    return path

# Satellite Map Generator with Hospitals and Rescue Centers
def generate_satellite_map():
    m = folium.Map(location=[22.5, 78.5], zoom_start=5)

    # Esri Satellite Layer
    folium.TileLayer(
        tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr='Esri Satellite',
        name='Satellite',
        overlay=False,
        control=True
    ).add_to(m)

    fg = folium.FeatureGroup(name="Disasters").add_to(m)

    # Disaster Markers
    for entry in disaster_data:
        color = severity_colors.get(entry["severity"], "gray")
        radius = severity_radius.get(entry["severity"], 3000)
        dist_km = round(geodesic((entry["lat"], entry["lon"]), (entry["safe_lat"], entry["safe_lon"])).km, 2)

        folium.Marker(
            location=[entry["lat"], entry["lon"]],
            tooltip=entry["city"],
            popup=folium.Popup(f"""
                <b>City:</b> {entry['city']}<br>
                <b>Severity:</b> <span style='color:{color}'>{entry['severity']}</span><br>
                <b>Safe Zone:</b> {entry['safe_zone']}<br>
                <b>Rescue Number:</b> <a href='tel:{entry['rescue_number']}'>{entry['rescue_number']}</a><br>
                <b>Distance to Safe Zone:</b> {dist_km} km
            """, max_width=300),
            icon=folium.Icon(color=color)
        ).add_to(fg)

        folium.Marker(
            location=[entry["safe_lat"], entry["safe_lon"]],
            tooltip=f"✅ {entry['safe_zone']}",
            popup=f"Safe Zone for {entry['city']}",
            icon=folium.Icon(color="green", icon="ok-sign")
        ).add_to(m)

        folium.PolyLine(
            locations=[[entry["lat"], entry["lon"]], [entry["safe_lat"], entry["safe_lon"]]],
            color="blue", weight=2, tooltip=f"{dist_km} km route"
        ).add_to(m)

        folium.Circle(
            location=[entry["lat"], entry["lon"]],
            radius=radius,
            color=color,
            fill=True,
            fill_opacity=0.2
        ).add_to(m)

    # Add Support Centers: Hospitals & Rescue Centers
    support_points = [
    {"type": "Hospital", "name": "Apollo Hospital Delhi", "lat": 28.5672, "lon": 77.2100},
    {"type": "Rescue Center", "name": "Delhi National Rescue", "lat": 28.6139, "lon": 77.2090},

    {"type": "Hospital", "name": "Mumbai Global Hospital", "lat": 19.0700, "lon": 72.8800},
    {"type": "Rescue Center", "name": "Mumbai Coastal Rescue", "lat": 19.0450, "lon": 72.8650},

    {"type": "Hospital", "name": "Chennai Medical Center", "lat": 13.0700, "lon": 80.2500},
    {"type": "Rescue Center", "name": "Chennai Cyclone Rescue HQ", "lat": 13.0600, "lon": 80.2700},

    {"type": "Hospital", "name": "Kolkata Relief Hospital", "lat": 22.5900, "lon": 88.3800},
    {"type": "Rescue Center", "name": "Kolkata Flood Rescue Base", "lat": 22.5850, "lon": 88.3950},

    {"type": "Hospital", "name": "Coimbatore Health Mission", "lat": 11.0100, "lon": 76.9600},
    {"type": "Rescue Center", "name": "Coimbatore Rescue Station", "lat": 11.0000, "lon": 76.9700}
]


    for point in support_points:
        if point["type"] == "Hospital":
            # Red medical cross icon using HTML styling
            icon_html = """
                <div style="
                    background-color: white;
                    border: 2px solid red;
                    border-radius: 50%;
                    width: 24px;
                    height: 24px;
                    text-align: center;
                    font-size: 18px;
                    font-weight: bold;
                    color: red;
                    line-height: 20px;
                ">+</div>
            """
            icon = folium.DivIcon(html=icon_html)
        else:
            icon = folium.Icon(color="cadetblue", icon="life-ring", prefix="fa")

        folium.Marker(
            location=[point["lat"], point["lon"]],
            tooltip=f"{point['type']}: {point['name']}",
            icon=icon
        ).add_to(m)

    # Search & Layers
    Search(layer=fg, search_label="tooltip", placeholder="Search city...", collapsed=False).add_to(m)
    folium.LayerControl().add_to(m)

    # Legend with Hospital/Rescue Symbol
    legend_html = '''
    <div style="position: fixed; bottom: 40px; left: 40px; z-index:9999; background-color: white; padding: 10px; border:2px solid grey;">
        <h4>Severity Legend</h4>
        <i style="background:darkred;width:15px;height:15px;display:inline-block;"></i> Critical<br>
        <i style="background:orange;width:15px;height:15px;display:inline-block;"></i> Severe<br>
        <i style="background:red;width:15px;height:15px;display:inline-block;"></i> High<br>
        <i style="background:blue;width:15px;height:15px;display:inline-block;"></i> Moderate<br><br>
        <b style="color:red;">+</b> Hospital<br>
        <i class="fa fa-life-ring" style="color:cadetblue;"></i> Rescue Center
    </div>
    '''
    m.get_root().html.add_child(folium.Element(legend_html))

    m.save("disaster_map.html")
    with open("disaster_map.html", "r", encoding="utf-8") as f:
        encoded_html = base64.b64encode(f.read().encode()).decode()

    return f'<iframe src="data:text/html;base64,{encoded_html}" width="100%" height="500" style="border-radius:12px;border:2px solid #ccc;"></iframe>'

# AI Simulator
def simulate_disaster_response(description):
    if not description.strip():
        return "❌ Please enter a disaster description.", None, None, None

    disaster_type = "Flood" if "flood" in description.lower() else "Earthquake"
    affected = random.randint(1000, 10000)
    resources = {
        "Medical": random.randint(10, 60),
        "Food": random.randint(30, 120),
        "Rescue Teams": random.randint(5, 35),
        "Shelter": random.randint(15, 80)
    }

    pie_path = generate_pie_chart(resources)
    bar_path = generate_bar_chart(resources)
    map_html = generate_satellite_map()

    summary = f"""
### 🧠 <span style='color:#1a73e8'><b>AI Response Summary</b></span>
📌 <b>Disaster Type:</b> {disaster_type}<br>
👥 <b>Estimated Affected People:</b> {affected}<br>
🚑 <b>Resources Allocated:</b><br>
""" + "<br>".join([f"- <b>{k}</b>: {v}" for k, v in resources.items()])

    return summary, pie_path, bar_path, map_html

# Gradio UI
with gr.Blocks(title="AI Disaster Response Simulator") as demo:
    gr.Markdown("<h1 style='text-align:center; color:#d62828;'>🌪️ AI Disaster Response Simulator</h1>")
    gr.Markdown("<p style='text-align:center;'>Generate satellite maps, evacuation plans, and resource stats</p>")

    with gr.Row():
        desc_input = gr.Textbox(label="📋 Enter Disaster Description", placeholder="e.g., Flood in Bihar affecting 5000 people")
        simulate_btn = gr.Button("🚨 Run Simulation")

    summary_output = gr.Markdown()
    pie_output = gr.Image(label="📊 Pie Chart: Resource Distribution")
    bar_output = gr.Image(label="📈 Bar Chart: Resource Quantities")
    map_output = gr.HTML(label="🛰️ Satellite Disaster Map")

    simulate_btn.click(
        simulate_disaster_response,
        inputs=[desc_input],
        outputs=[summary_output, pie_output, bar_output, map_output]
    )

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ee15155bee719e14ea.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
